<a href="https://colab.research.google.com/github/aruaru0/Foursquare-Location-Matching/blob/main/FourSqure_CV%E7%A2%BA%E8%AA%8D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# for colab

In [ ]:
import sys
is_colab = False
if "google.colab" in sys.modules:
  is_colab = True

In [ ]:
if is_colab:
  from google.colab import drive
  drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
if is_colab:
  !mkdir -p ~/.kaggle
  !cp /content/drive/MyDrive/datas/kaggle.json  ~/.kaggle/
  !chmod 600 /root/.kaggle/kaggle.json

In [ ]:
if is_colab:
  !pip install kaggle
  !apt install unzip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-21ubuntu1.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
if is_colab:
  !mkdir input output

In [ ]:
if is_colab:
  !kaggle competitions download -c foursquare-location-matching
  !unzip foursquare-location-matching.zip -d input/foursquare-location-matching

 96% 152M/159M [00:01<00:00, 93.7MB/s]
100% 159M/159M [00:01<00:00, 102MB/s] 
Archive:  foursquare-location-matching.zip
  inflating: input/foursquare-location-matching/pairs.csv  
  inflating: input/foursquare-location-matching/sample_submission.csv  
  inflating: input/foursquare-location-matching/test.csv  
  inflating: input/foursquare-location-matching/train.csv  


In [ ]:
if is_colab:
  !kaggle datasets download -d guoyonfan/binary-lgb-baseline
  !unzip /content/binary-lgb-baseline.zip -d input/binary-lgb-baseline

  0% 0.00/3.90M [00:00<?, ?B/s]
100% 3.90M/3.90M [00:00<00:00, 47.1MB/s]
Archive:  /content/binary-lgb-baseline.zip
  inflating: input/binary-lgb-baseline/improved_lgb_baseline.lgb  
  inflating: input/binary-lgb-baseline/lgb_baseline.lgb  


In [ ]:
if is_colab:
  !cp input/binary-lgb-baseline/*  /content/drive/MyDrive/datas/foursquare

In [ ]:
if is_colab:
  !pip install Levenshtein
  !pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# change directory

In [ ]:
if is_colab:
  %cd /content/output

/content/output


In [ ]:
if is_colab:
    !pip install pykakasi
    !pip install fuzzywuzzy
else :
    !unzip ../input/foursquare-lightgbm-trained/pykakasi_deps.dontopenthiskaggle/pykakasi_deps.dontopenthiskaggle -d .
    !conda install ./pykakasi_deps/offline_pykakasi.tar.bz2
    !conda install ./pykakasi_deps/offline_jaconv.tar.bz2
    !conda install ./pykakasi_deps/offline_deprecated.tar.bz2
    !pip install ../input/foursquare-lightgbm-trained/fuzzywuzzy-0.18.0-py2.py3-none-any.whl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### This notebook is the improved version of [binary_lgb_baseline](https://www.kaggle.com/code/guoyonfan/binary-lgb-baseline-0-834), and mainly benefits from the [simple_recall_method](https://www.kaggle.com/code/guoyonfan/simple-recall-method). This notebook improves the LB score of the binary_lgb_baseline from 0.834 to 0.839.

### If you find these notebooks useful, please UPVOTE!

## restart here to check

In [ ]:
import sys
is_colab = False
if "google.colab" in sys.modules:
  is_colab = True

if is_colab:
  %cd /content/output

/content/output


In [ ]:
## Imports
import warnings
warnings.filterwarnings('ignore')

import os
import gc
import time
import random
import Levenshtein
import difflib
import multiprocessing
import pandas as pd
import numpy as np
import lightgbm as lgb
from collections import Counter
from tqdm.auto import tqdm
from tqdm.notebook import tqdm

tqdm.pandas()

from sklearn.neighbors import KNeighborsRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

from catboost import CatBoostClassifier
import pykakasi

from fuzzywuzzy import fuzz
import math

In [ ]:
TRAIN_FEATURES  = ['kneighbors', 'kneighbors_country',
       'kneighbors_categories', 'categories_index', 'latdiff',
       'londiff', 'manhattan', 'euclidean', 'haversine', 'name_sim',
       'name_gesh', 'name_leven', 'name_jaro', 'name_lcs', 'name_fuz',
       'name_len_diff', 'name_nleven', 'name_nlcsk', 'name_nlcs',
       'address_sim', 'address_gesh', 'address_leven', 'address_jaro',
       'address_lcs', 'address_fuz', 'address_len_diff', 'address_nleven',
       'address_nlcsk', 'address_nlcs', 'city_gesh', 'city_leven', 'city_jaro',
       'city_lcs', 'city_fuz', 'city_len_diff', 'city_nleven', 'city_nlcsk',
       'city_nlcs', 'state_sim', 'state_gesh', 'state_leven', 'state_jaro',
       'state_lcs', 'state_fuz', 'state_len_diff', 'state_nleven',
       'zip_gesh', 'zip_jaro',
       'zip_lcs', 'url_sim',
       'url_lcs', 'phone_gesh', 'phone_jaro', 'phone_lcs',
       'categories_sim', 'categories_gesh', 'categories_leven',
       'categories_jaro', 'categories_lcs', 'categories_fuz',
       'categories_len_diff', 'categories_nleven', 'categories_nlcsk',
       'categories_nlcs',
       'categories2_sim',
       'categories2_gesh', 'categories2_leven', 'categories2_jaro',
       'categories2_lcs', 'categories2_fuz', 'categories2_len_diff',
       'categories2_nleven', 'categories2_nlcsk', 'categories2_nlcs']

In [ ]:
## Parameters
num_neighbors = 10
num_neighbors2 = 6
num_neighbors3 = 22
# num_neighbors = 10
# num_neighbors2 = 5
# num_neighbors3 = 22
SEED = 2022
THRESHOLD = 0.5
NUM_SPLIT = 10
feat_columns = ['dist', 'name', 'address', 'city', 
            'state', 'zip', 'url', 
           'phone', 'categories', 'country', 'categories2']
vec_columns = ['name', 'categories', 'address', 
               'state', 'url', 'country', 'categories2']
rec_columns = ['name', 'address', 'categories', 'address', 'phone']

def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    
seed_everything(SEED)

In [ ]:
sel_cat = {'ATMs': 221,
 'Accessories Stores': 225,
 'Advertising Agencies': 250,
 'Airport Lounges': 315,
 'Airport Services': 35,
 'Airport Terminals': 140,
 'Airports': 364,
 'American Restaurants': 117,
 'Animal Shelters': 259,
 'Antique Shops': 476,
 'Aquariums': 358,
 'Arcades': 178,
 'Argentinian Restaurants': 411,
 'Art Galleries': 168,
 'Art Museums': 299,
 'Arts & Crafts Stores': 240,
 'Asian Restaurants': 443,
 'Asian Restaurants, Chinese Restaurants': 429,
 'Assisted Living': 419,
 'Athletics & Sports': 98,
 'Auditoriums': 13,
 'Auto Dealerships': 367,
 'Auto Dealerships, Automotive Shops': 62,
 'Auto Garages': 381,
 'Auto Workshops': 258,
 'Auto Workshops, Automotive Shops': 112,
 'Auto Workshops, Automotive Shops, Auto Garages': 488,
 'Automotive Shops': 227,
 'Automotive Shops, Auto Dealerships': 413,
 'Automotive Shops, Auto Garages': 97,
 'Automotive Shops, Gas Stations': 331,
 'BBQ Joints': 201,
 'Baby Stores': 77,
 'Bagel Shops': 55,
 'Baggage Claims': 160,
 'Bakeries': 438,
 'Bakeries, Cafés': 433,
 'Banks': 449,
 'Banks, ATMs': 100,
 'Banks, Buildings': 339,
 'Banks, Financial or Legal Services': 33,
 'Banks, Offices': 388,
 'Bars': 378,
 'Baseball Fields': 457,
 'Baseball Stadiums': 83,
 'Basketball Courts': 254,
 'Beach Bars': 61,
 'Beaches': 1,
 'Beaches, Surf Spots': 296,
 'Bed & Breakfasts': 451,
 'Beer Bars': 348,
 'Beer Gardens': 387,
 'Big Box Stores': 389,
 'Big Box Stores, Department Stores': 492,
 'Big Box Stores, Supermarkets': 88,
 'Big Box Stores, Supermarkets, Grocery Stores': 385,
 'Bike Rentals / Bike Shares': 276,
 'Bike Shops': 200,
 'Bistros': 177,
 'Boarding Houses': 41,
 'Boats or Ferries': 495,
 'Bookstores': 70,
 'Border Crossings': 114,
 'Botanical Gardens': 414,
 'Boutiques': 182,
 'Bowling Alleys': 327,
 'Brazilian Restaurants': 147,
 'Breakfast Spots': 337,
 'Breweries': 156,
 'Bridal Shops': 466,
 'Bridges': 261,
 'Bubble Tea Shops': 215,
 'Buddhist Temples': 118,
 'Buffets': 132,
 'Buildings': 134,
 'Buildings, Offices': 213,
 'Buildings, Residential Buildings (Apartments / Condos)': 104,
 'Burger Joints': 408,
 'Burger Joints, Fast Food Restaurants': 22,
 'Burrito Places': 345,
 'Bus Stations': 67,
 'Bus Stops': 111,
 'Business Centers': 382,
 'Business Services': 349,
 'Butchers': 226,
 'Cafeterias': 57,
 'Cafés': 359,
 'Cafés, Bars': 116,
 'Cafés, Breakfast Spots': 51,
 'Cafés, Coffee Shops': 304,
 'Cafés, Restaurants': 184,
 'Camera Stores': 437,
 'Campaign Offices': 371,
 'Campgrounds': 308,
 'Candy Stores': 235,
 'Capitol Buildings': 290,
 'Car Washes': 236,
 'Caribbean Restaurants': 74,
 'Casinos': 167,
 'Castles': 248,
 'Cemeteries': 369,
 'Chinese Restaurants': 141,
 'Chinese Restaurants, Asian Restaurants': 496,
 'Chiropractors': 15,
 'Chocolate Shops': 44,
 'Churches': 155,
 'City Halls': 412,
 'Clothing Stores': 8,
 'Clothing Stores, Boutiques': 302,
 'Clothing Stores, Department Stores': 319,
 "Clothing Stores, Women's Stores": 123,
 'Club Houses': 56,
 'Cocktail Bars': 12,
 'Coffee Shops': 145,
 'Coffee Shops, Cafés': 247,
 'College Academic Buildings': 48,
 'College Academic Buildings, College Classrooms': 281,
 'College Administrative Buildings': 372,
 'College Arts Buildings': 410,
 'College Auditoriums': 303,
 'College Bookstores': 264,
 'College Cafeterias': 442,
 'College Classrooms': 343,
 'College Communications Buildings': 499,
 'College Engineering Buildings': 405,
 'College Gyms': 375,
 'College Labs': 374,
 'College Libraries': 24,
 'College Quads': 464,
 'College Rec Centers': 288,
 'College Residence Halls': 144,
 'College Science Buildings': 353,
 'College Technology Buildings': 38,
 'College Theaters': 282,
 'Colleges & Universities': 300,
 'Comedy Clubs': 9,
 'Comfort Food Restaurants': 210,
 'Community Centers': 192,
 'Community Colleges': 298,
 'Concert Halls': 202,
 'Conference Rooms': 189,
 'Construction & Landscaping': 244,
 'Convenience Stores': 37,
 'Convenience Stores, Gas Stations': 18,
 'Convenience Stores, Snack Places': 28,
 'Convention Centers': 273,
 'Corporate Cafeterias': 365,
 'Cosmetics Shops': 377,
 'Courthouses': 274,
 'Coworking Spaces': 3,
 'Credit Unions': 42,
 'Cultural Centers': 231,
 'Cupcake Shops': 486,
 'Dance Studios': 222,
 'Daycares': 432,
 'Delis / Bodegas': 356,
 "Dentist's Offices": 209,
 'Department Stores': 75,
 'Department Stores, Clothing Stores': 85,
 'Department Stores, Clothing Stores, Boutiques': 4,
 'Design Studios': 262,
 'Dessert Shops': 162,
 'Dessert Shops, Cafés': 366,
 'Dim Sum Restaurants': 64,
 'Diners': 95,
 'Discount Stores': 220,
 'Discount Stores, Arts & Crafts Stores, Convenience Stores': 174,
 'Distribution Centers': 82,
 'Dive Bars': 84,
 "Doctor's Offices": 291,
 'Dog Runs': 471,
 'Doner Restaurants': 173,
 'Donut Shops': 491,
 'Donut Shops, Coffee Shops': 383,
 'Driving Schools': 21,
 'Drugstores': 415,
 'Dumpling Restaurants': 2,
 'Eastern European Restaurants': 422,
 'Electronics Stores': 434,
 'Electronics Stores, Mobile Phone Shops': 179,
 'Elementary Schools': 380,
 'Embassies / Consulates': 7,
 'Emergency Rooms': 334,
 'Ethiopian Restaurants': 26,
 'Event Spaces': 293,
 'Eye Doctors': 436,
 'Factories': 421,
 'Fairs': 404,
 'Farmers Markets': 463,
 'Farms': 161,
 'Fast Food Restaurants': 395,
 'Fast Food Restaurants, Burger Joints': 474,
 'Fast Food Restaurants, Fried Chicken Joints': 401,
 'Festivals': 54,
 'Fields': 493,
 'Filipino Restaurants': 19,
 'Financial or Legal Services': 267,
 'Fire Stations': 125,
 'Fish & Chips Shops': 11,
 'Fish Markets': 351,
 'Flea Markets': 138,
 'Flower Shops': 108,
 'Food': 79,
 'Food & Drink Shops': 129,
 'Food Courts': 120,
 'Food Stands': 246,
 'Food Trucks': 150,
 'Football Stadiums': 27,
 'Forests': 431,
 'Fraternity Houses': 313,
 'French Restaurants': 126,
 'Fried Chicken Joints': 468,
 'Fried Chicken Joints, Fast Food Restaurants': 297,
 'Friteries': 23,
 'Frozen Yogurt Shops': 482,
 'Fruit & Vegetable Stores': 105,
 'Funeral Homes': 420,
 'Furniture / Home Stores': 175,
 'Gaming Cafes': 205,
 'Garden Centers': 355,
 'Gardens': 31,
 'Gas Stations': 78,
 'Gas Stations, Convenience Stores': 107,
 'Gastropubs': 456,
 'Gay Bars': 346,
 'General Colleges & Universities': 483,
 'General Entertainment': 498,
 'General Travel': 139,
 'German Restaurants': 396,
 'Gift Shops': 322,
 'Golf Courses': 330,
 'Gourmet Shops': 323,
 'Government Buildings': 424,
 'Government Buildings, Offices': 65,
 'Greek Restaurants': 386,
 'Grocery Stores': 376,
 'Grocery Stores, Convenience Stores': 30,
 'Grocery Stores, Supermarkets': 305,
 'Gyms': 124,
 'Gyms or Fitness Centers': 317,
 'Gyms or Fitness Centers, Gyms': 255,
 'Gyms, Gyms or Fitness Centers': 361,
 'Harbors / Marinas': 218,
 'Hardware Stores': 400,
 'Hardware Stores, Garden Centers': 439,
 'Health & Beauty Services': 328,
 'Health Food Stores': 350,
 'High Schools': 278,
 'Historic Sites': 277,
 'History Museums': 96,
 'Hobby Shops': 122,
 'Home Services & Repairs': 128,
 'Hookah Bars': 53,
 'Hospital Wards': 447,
 'Hospitals': 237,
 'Hospitals, Medical Centers': 50,
 'Hostels': 0,
 'Hot Dog Joints': 152,
 'Hot Springs': 68,
 'Hotel Bars': 133,
 'Hotels': 211,
 'Hotels, Bed & Breakfasts': 306,
 'Hotels, Resorts': 478,
 'Hotpot Restaurants': 127,
 'Housing Developments': 271,
 'Housing Developments, Residential Buildings (Apartments / Condos)': 198,
 'IT Services': 180,
 'Ice Cream Shops': 344,
 'Ice Cream Shops, Dessert Shops': 39,
 'Indian Restaurants': 172,
 'Indie Movie Theaters': 45,
 'Indonesian Restaurants': 142,
 'Industrial Estates': 428,
 'Insurance Offices': 228,
 'Internet Cafes': 430,
 'Italian Restaurants': 188,
 'Italian Restaurants, Pizza Places': 458,
 'Japanese Curry Restaurants': 487,
 'Japanese Restaurants': 212,
 'Japanese Restaurants, Sushi Restaurants': 199,
 'Jewelry Stores': 336,
 'Juice Bars': 453,
 'Karaoke Bars': 229,
 'Kebab Restaurants': 143,
 'Kids Stores': 176,
 'Korean Restaurants': 268,
 'Lakes': 325,
 'Language Schools': 203,
 'Laundromats': 241,
 'Laundry Services': 87,
 'Law Schools': 47,
 'Lawyers': 416,
 'Libraries': 462,
 'Light Rail Stations': 196,
 'Lighthouses': 473,
 'Lingerie Stores': 214,
 "Lingerie Stores, Women's Stores": 206,
 'Liquor Stores': 135,
 'Locksmiths': 73,
 'Lounges': 379,
 'Malay Restaurants': 316,
 'Markets': 342,
 'Martial Arts Schools': 340,
 'Massage Studios': 151,
 'Mattress Stores': 251,
 'Medical Centers': 394,
 "Medical Centers, Doctor's Offices": 472,
 'Medical Centers, Hospitals': 80,
 'Medical Labs': 479,
 'Medical Schools': 266,
 'Mediterranean Restaurants': 260,
 'Meeting Rooms': 263,
 "Men's Stores": 121,
 'Mental Health Offices': 280,
 'Metro Stations': 332,
 'Mexican Restaurants': 368,
 'Middle Eastern Restaurants': 338,
 'Middle Schools': 295,
 'Military Bases': 32,
 'Miscellaneous Shops': 398,
 'Mobile Phone Shops': 43,
 'Mobile Phone Shops, Electronics Stores': 103,
 'Monuments / Landmarks': 363,
 'Mosques': 109,
 'Motels': 157,
 'Motorcycle Shops': 69,
 'Mountains': 475,
 'Movie Theaters': 450,
 'Movie Theaters, Multiplexes': 326,
 'Multiplexes': 253,
 'Multiplexes, Movie Theaters': 10,
 'Museums': 90,
 'Music Festivals': 163,
 'Music Stores': 92,
 'Music Venues': 301,
 'Nail Salons': 320,
 'National Parks': 409,
 'Nature Preserves': 307,
 'New American Restaurants': 390,
 'Newsstands': 384,
 'Nightclubs': 448,
 'Non-Profits': 81,
 'Noodle Houses': 490,
 'Nursery Schools': 406,
 'Offices': 370,
 'Offices, Buildings': 185,
 'Offices, Coworking Spaces': 275,
 'Optical Shops': 20,
 'Optical Shops, Eye Doctors': 36,
 'Other Events': 279,
 'Other Great Outdoors': 193,
 'Other Nightlife': 137,
 'Other Repair Shops': 245,
 'Paper / Office Supplies Stores': 314,
 'Parking': 194,
 'Parks': 60,
 'Parks, Playgrounds': 270,
 'Pastry Shops': 234,
 'Pedestrian Plazas': 454,
 'Performing Arts Venues': 481,
 'Pet Services': 393,
 'Pet Stores': 49,
 'Pharmacies': 186,
 'Pharmacies, Drugstores': 89,
 'Photography Labs': 204,
 'Photography Studios': 242,
 'Photography Studios, Photography Labs': 310,
 'Physical Therapists': 459,
 'Piers': 347,
 'Pizza Places': 146,
 'Pizza Places, Italian Restaurants': 265,
 'Pizza Places, Restaurants': 148,
 'Playgrounds': 362,
 'Plazas': 219,
 'Police Stations': 29,
 'Pool Halls': 216,
 'Pools': 469,
 'Portuguese Restaurants': 52,
 'Post Offices': 239,
 'Print Shops': 170,
 'Private Schools': 435,
 'Professional & Other Places': 294,
 'Public Art': 477,
 'Pubs': 341,
 'Racetracks': 256,
 'Radio Stations': 311,
 'Ramen Restaurants': 159,
 'Real Estate Offices': 318,
 'Record Shops': 181,
 'Recreation Centers': 257,
 'Recycling Facilities': 309,
 'Rental Car Locations': 165,
 'Residential Buildings (Apartments / Condos)': 391,
 'Residential Buildings (Apartments / Condos), Buildings': 40,
 'Residential Buildings (Apartments / Condos), Housing Developments': 6,
 'Resorts': 243,
 'Resorts, Hotels': 321,
 'Rest Areas': 289,
 'Restaurants': 426,
 'Restaurants, Cafés': 154,
 'Rivers': 357,
 'Rock Clubs': 131,
 'Sake Bars': 397,
 'Salad Places': 407,
 'Salons / Barbershops': 232,
 'Salons / Barbershops, Health & Beauty Services': 333,
 'Salons / Barbershops, Nail Salons': 16,
 'Sandwich Places': 17,
 'Sandwich Places, Fast Food Restaurants': 283,
 'Sandwich Places, Fast Food Restaurants, Restaurants': 183,
 'Scenic Lookouts': 195,
 'Schools': 101,
 'Seafood Restaurants': 34,
 'Shipping Stores': 63,
 'Shipping Stores, Business Services': 470,
 'Shoe Stores': 233,
 'Shopping Malls': 136,
 'Shopping Malls, Department Stores': 418,
 'Shopping Plazas': 360,
 'Shops & Services': 403,
 'Shrines': 238,
 'Ski Areas': 5,
 'Smoke Shops': 99,
 'Snack Places': 102,
 'Soba Restaurants': 417,
 'Soccer Fields': 230,
 'Soccer Stadiums': 59,
 'Social Clubs': 452,
 'Som Tum Restaurants': 171,
 'Sorority Houses': 191,
 'Soup Places': 158,
 'Spanish Restaurants': 91,
 'Spas': 113,
 'Speakeasies': 207,
 'Spiritual Centers': 460,
 'Sporting Goods Shops': 223,
 'Sporting Goods Shops, Clothing Stores': 110,
 'Sports Bars': 130,
 'Sports Clubs': 66,
 'Stables': 312,
 'Stadiums': 252,
 'Steakhouses': 446,
 'Storage Facilities': 373,
 'Street Fairs': 166,
 'Strip Clubs': 94,
 'Student Centers': 190,
 'Supermarkets': 208,
 'Supermarkets, Discount Stores': 187,
 'Supermarkets, Grocery Stores': 465,
 'Surf Spots': 285,
 'Sushi Restaurants': 119,
 'Sushi Restaurants, Japanese Restaurants': 93,
 'TV Stations': 480,
 'Taco Places': 269,
 'Tailor Shops': 71,
 'Tanning Salons': 425,
 'Tapas Restaurants': 444,
 'Tattoo Parlors': 72,
 'Tea Rooms': 25,
 'Tech Startups': 484,
 'Temples': 169,
 'Tennis Courts': 14,
 'Thai Restaurants': 249,
 'Theaters': 272,
 'Theme Park Rides/Attractions': 76,
 'Theme Parks': 352,
 'Thrift / Vintage Stores': 58,
 'Tourist Information Centers': 149,
 'Toy / Game Stores': 441,
 'Trade Schools': 86,
 'Trails': 489,
 'Train Stations': 392,
 'Tram Stations': 224,
 'Transportation Services': 445,
 'Travel Agencies': 461,
 'Travel Lounges': 335,
 'Tunnels': 164,
 'Turkish Restaurants': 399,
 'Udon Restaurants': 106,
 'Universities': 440,
 'Urgent Care Centers': 329,
 'Vacation Rentals': 287,
 'Vegetarian / Vegan Restaurants': 292,
 'Veterinarians': 153,
 'Video Game Stores': 423,
 'Video Stores': 284,
 'Vietnamese Restaurants': 485,
 'Warehouse Stores': 402,
 'Warehouses': 286,
 'Water Parks': 354,
 'Waterfalls': 455,
 'Waterfronts': 217,
 'Wedding Halls': 46,
 'Wine Bars': 197,
 'Wine Shops': 494,
 'Wineries': 324,
 'Wings Joints': 497,
 "Women's Stores": 427,
 'Yoga Studios': 467,
 'Zoos': 115}

In [ ]:
# for traindata4
# sel_cat = {'ATMs': 444,
#  'Accessories Stores': 270,
#  'Advertising Agencies': 248,
#  'Airport Lounges': 178,
#  'Airport Services': 176,
#  'Airport Terminals': 490,
#  'Airports': 376,
#  'American Restaurants': 262,
#  'Animal Shelters': 415,
#  'Antique Shops': 456,
#  'Aquariums': 435,
#  'Arcades': 482,
#  'Argentinian Restaurants': 403,
#  'Art Galleries': 29,
#  'Art Museums': 373,
#  'Arts & Crafts Stores': 368,
#  'Asian Restaurants': 185,
#  'Asian Restaurants, Chinese Restaurants': 372,
#  'Assisted Living': 374,
#  'Athletics & Sports': 208,
#  'Auditoriums': 232,
#  'Auto Dealerships': 154,
#  'Auto Dealerships, Automotive Shops': 184,
#  'Auto Garages': 165,
#  'Auto Workshops': 273,
#  'Auto Workshops, Automotive Shops': 466,
#  'Auto Workshops, Automotive Shops, Auto Garages': 391,
#  'Automotive Shops': 10,
#  'Automotive Shops, Auto Dealerships': 155,
#  'Automotive Shops, Auto Garages': 5,
#  'Automotive Shops, Gas Stations': 440,
#  'BBQ Joints': 491,
#  'Baby Stores': 38,
#  'Bagel Shops': 175,
#  'Baggage Claims': 494,
#  'Bakeries': 453,
#  'Bakeries, Cafés': 123,
#  'Banks': 496,
#  'Banks, ATMs': 498,
#  'Banks, Buildings': 419,
#  'Banks, Financial or Legal Services': 130,
#  'Banks, Offices': 370,
#  'Bars': 151,
#  'Baseball Fields': 332,
#  'Baseball Stadiums': 287,
#  'Basketball Courts': 150,
#  'Beach Bars': 302,
#  'Beaches': 258,
#  'Beaches, Surf Spots': 463,
#  'Bed & Breakfasts': 306,
#  'Beer Bars': 245,
#  'Beer Gardens': 499,
#  'Big Box Stores': 272,
#  'Big Box Stores, Department Stores': 52,
#  'Big Box Stores, Supermarkets': 402,
#  'Big Box Stores, Supermarkets, Grocery Stores': 428,
#  'Bike Rentals / Bike Shares': 196,
#  'Bike Shops': 355,
#  'Bistros': 337,
#  'Boarding Houses': 161,
#  'Boats or Ferries': 421,
#  'Bookstores': 153,
#  'Border Crossings': 425,
#  'Botanical Gardens': 328,
#  'Boutiques': 293,
#  'Bowling Alleys': 80,
#  'Brazilian Restaurants': 179,
#  'Breakfast Spots': 105,
#  'Breweries': 369,
#  'Bridal Shops': 495,
#  'Bridges': 437,
#  'Bubble Tea Shops': 330,
#  'Buddhist Temples': 139,
#  'Buffets': 384,
#  'Buildings': 17,
#  'Buildings, Offices': 426,
#  'Buildings, Residential Buildings (Apartments / Condos)': 348,
#  'Burger Joints': 189,
#  'Burger Joints, Fast Food Restaurants': 310,
#  'Burrito Places': 480,
#  'Bus Stations': 44,
#  'Bus Stops': 129,
#  'Business Centers': 49,
#  'Business Services': 290,
#  'Butchers': 41,
#  'Cafeterias': 392,
#  'Cafés': 497,
#  'Cafés, Bars': 174,
#  'Cafés, Breakfast Spots': 452,
#  'Cafés, Coffee Shops': 271,
#  'Cafés, Restaurants': 410,
#  'Camera Stores': 57,
#  'Campaign Offices': 116,
#  'Campgrounds': 218,
#  'Candy Stores': 279,
#  'Capitol Buildings': 458,
#  'Car Washes': 324,
#  'Caribbean Restaurants': 301,
#  'Casinos': 268,
#  'Castles': 133,
#  'Cemeteries': 148,
#  'Chinese Restaurants': 144,
#  'Chinese Restaurants, Asian Restaurants': 215,
#  'Chiropractors': 352,
#  'Chocolate Shops': 280,
#  'Churches': 343,
#  'City Halls': 242,
#  'Clothing Stores': 387,
#  'Clothing Stores, Boutiques': 112,
#  'Clothing Stores, Department Stores': 397,
#  "Clothing Stores, Women's Stores": 422,
#  'Club Houses': 241,
#  'Cocktail Bars': 334,
#  'Coffee Shops': 371,
#  'Coffee Shops, Cafés': 36,
#  'College Academic Buildings': 489,
#  'College Academic Buildings, College Classrooms': 60,
#  'College Administrative Buildings': 350,
#  'College Arts Buildings': 264,
#  'College Auditoriums': 180,
#  'College Bookstores': 441,
#  'College Cafeterias': 7,
#  'College Classrooms': 15,
#  'College Communications Buildings': 237,
#  'College Engineering Buildings': 226,
#  'College Gyms': 103,
#  'College Labs': 31,
#  'College Libraries': 284,
#  'College Quads': 96,
#  'College Rec Centers': 274,
#  'College Residence Halls': 390,
#  'College Science Buildings': 260,
#  'College Technology Buildings': 51,
#  'College Theaters': 93,
#  'Colleges & Universities': 43,
#  'Comedy Clubs': 198,
#  'Comfort Food Restaurants': 414,
#  'Community Centers': 288,
#  'Community Colleges': 79,
#  'Concert Halls': 1,
#  'Conference Rooms': 448,
#  'Construction & Landscaping': 338,
#  'Convenience Stores': 298,
#  'Convenience Stores, Gas Stations': 375,
#  'Convenience Stores, Snack Places': 187,
#  'Convention Centers': 67,
#  'Corporate Cafeterias': 204,
#  'Cosmetics Shops': 72,
#  'Courthouses': 78,
#  'Coworking Spaces': 71,
#  'Credit Unions': 382,
#  'Cultural Centers': 333,
#  'Cupcake Shops': 118,
#  'Dance Studios': 408,
#  'Daycares': 236,
#  'Delis / Bodegas': 100,
#  "Dentist's Offices": 339,
#  'Department Stores': 467,
#  'Department Stores, Clothing Stores': 219,
#  'Department Stores, Clothing Stores, Boutiques': 481,
#  'Design Studios': 447,
#  'Dessert Shops': 256,
#  'Dessert Shops, Cafés': 347,
#  'Dim Sum Restaurants': 217,
#  'Diners': 401,
#  'Discount Stores': 6,
#  'Discount Stores, Arts & Crafts Stores, Convenience Stores': 429,
#  'Distribution Centers': 321,
#  'Dive Bars': 493,
#  "Doctor's Offices": 166,
#  'Dog Runs': 418,
#  'Doner Restaurants': 143,
#  'Donut Shops': 263,
#  'Donut Shops, Coffee Shops': 27,
#  'Driving Schools': 432,
#  'Drugstores': 487,
#  'Dumpling Restaurants': 18,
#  'Eastern European Restaurants': 253,
#  'Electronics Stores': 356,
#  'Electronics Stores, Mobile Phone Shops': 377,
#  'Elementary Schools': 140,
#  'Embassies / Consulates': 286,
#  'Emergency Rooms': 475,
#  'Ethiopian Restaurants': 69,
#  'Event Spaces': 289,
#  'Eye Doctors': 278,
#  'Factories': 299,
#  'Fairs': 450,
#  'Farmers Markets': 181,
#  'Farms': 222,
#  'Fast Food Restaurants': 308,
#  'Fast Food Restaurants, Burger Joints': 102,
#  'Fast Food Restaurants, Fried Chicken Joints': 191,
#  'Festivals': 3,
#  'Fields': 111,
#  'Filipino Restaurants': 367,
#  'Financial or Legal Services': 115,
#  'Fire Stations': 477,
#  'Fish & Chips Shops': 21,
#  'Fish Markets': 2,
#  'Flea Markets': 127,
#  'Flower Shops': 319,
#  'Food': 427,
#  'Food & Drink Shops': 233,
#  'Food Courts': 468,
#  'Food Stands': 239,
#  'Food Trucks': 98,
#  'Football Stadiums': 209,
#  'Forests': 24,
#  'Fraternity Houses': 325,
#  'French Restaurants': 383,
#  'Fried Chicken Joints': 0,
#  'Fried Chicken Joints, Fast Food Restaurants': 141,
#  'Friteries': 442,
#  'Frozen Yogurt Shops': 365,
#  'Fruit & Vegetable Stores': 269,
#  'Funeral Homes': 409,
#  'Furniture / Home Stores': 128,
#  'Gaming Cafes': 341,
#  'Garden Centers': 228,
#  'Gardens': 42,
#  'Gas Stations': 484,
#  'Gas Stations, Convenience Stores': 229,
#  'Gastropubs': 492,
#  'Gay Bars': 380,
#  'General Colleges & Universities': 344,
#  'General Entertainment': 353,
#  'General Travel': 136,
#  'German Restaurants': 461,
#  'Gift Shops': 423,
#  'Golf Courses': 212,
#  'Gourmet Shops': 125,
#  'Government Buildings': 85,
#  'Government Buildings, Offices': 194,
#  'Greek Restaurants': 320,
#  'Grocery Stores': 243,
#  'Grocery Stores, Convenience Stores': 234,
#  'Grocery Stores, Supermarkets': 182,
#  'Gyms': 340,
#  'Gyms or Fitness Centers': 81,
#  'Gyms or Fitness Centers, Gyms': 438,
#  'Gyms, Gyms or Fitness Centers': 63,
#  'Harbors / Marinas': 335,
#  'Hardware Stores': 379,
#  'Hardware Stores, Garden Centers': 45,
#  'Health & Beauty Services': 192,
#  'Health Food Stores': 76,
#  'High Schools': 311,
#  'Historic Sites': 446,
#  'History Museums': 351,
#  'Hobby Shops': 22,
#  'Home Services & Repairs': 216,
#  'Hookah Bars': 295,
#  'Hospital Wards': 345,
#  'Hospitals': 147,
#  'Hospitals, Medical Centers': 37,
#  'Hostels': 90,
#  'Hot Dog Joints': 113,
#  'Hot Springs': 12,
#  'Hotel Bars': 95,
#  'Hotels': 314,
#  'Hotels, Bed & Breakfasts': 34,
#  'Hotels, Resorts': 190,
#  'Hotpot Restaurants': 66,
#  'Housing Developments': 297,
#  'Housing Developments, Residential Buildings (Apartments / Condos)': 188,
#  'IT Services': 91,
#  'Ice Cream Shops': 478,
#  'Ice Cream Shops, Dessert Shops': 224,
#  'Indian Restaurants': 120,
#  'Indie Movie Theaters': 457,
#  'Indonesian Restaurants': 381,
#  'Industrial Estates': 99,
#  'Insurance Offices': 82,
#  'Internet Cafes': 349,
#  'Italian Restaurants': 126,
#  'Italian Restaurants, Pizza Places': 326,
#  'Japanese Curry Restaurants': 240,
#  'Japanese Restaurants': 455,
#  'Japanese Restaurants, Sushi Restaurants': 451,
#  'Jewelry Stores': 244,
#  'Juice Bars': 283,
#  'Karaoke Bars': 137,
#  'Kebab Restaurants': 227,
#  'Kids Stores': 168,
#  'Korean Restaurants': 183,
#  'Lakes': 9,
#  'Language Schools': 388,
#  'Laundromats': 158,
#  'Laundry Services': 70,
#  'Law Schools': 199,
#  'Lawyers': 203,
#  'Libraries': 68,
#  'Light Rail Stations': 316,
#  'Lighthouses': 23,
#  'Lingerie Stores': 386,
#  "Lingerie Stores, Women's Stores": 462,
#  'Liquor Stores': 200,
#  'Locksmiths': 33,
#  'Lounges': 146,
#  'Malay Restaurants': 246,
#  'Markets': 331,
#  'Martial Arts Schools': 378,
#  'Massage Studios': 413,
#  'Mattress Stores': 152,
#  'Medical Centers': 162,
#  "Medical Centers, Doctor's Offices": 336,
#  'Medical Centers, Hospitals': 474,
#  'Medical Labs': 400,
#  'Medical Schools': 309,
#  'Mediterranean Restaurants': 48,
#  'Meeting Rooms': 104,
#  "Men's Stores": 53,
#  'Mental Health Offices': 64,
#  'Metro Stations': 47,
#  'Mexican Restaurants': 361,
#  'Middle Eastern Restaurants': 359,
#  'Middle Schools': 114,
#  'Military Bases': 433,
#  'Miscellaneous Shops': 117,
#  'Mobile Phone Shops': 473,
#  'Mobile Phone Shops, Electronics Stores': 201,
#  'Monuments / Landmarks': 305,
#  'Mosques': 251,
#  'Motels': 300,
#  'Motorcycle Shops': 364,
#  'Mountains': 412,
#  'Movie Theaters': 169,
#  'Movie Theaters, Multiplexes': 211,
#  'Multiplexes': 464,
#  'Multiplexes, Movie Theaters': 55,
#  'Museums': 88,
#  'Music Festivals': 407,
#  'Music Stores': 50,
#  'Music Venues': 16,
#  'Nail Salons': 303,
#  'National Parks': 285,
#  'Nature Preserves': 221,
#  'New American Restaurants': 65,
#  'Newsstands': 360,
#  'Nightclubs': 235,
#  'Non-Profits': 206,
#  'Noodle Houses': 479,
#  'Nursery Schools': 266,
#  'Offices': 261,
#  'Offices, Buildings': 157,
#  'Offices, Coworking Spaces': 220,
#  'Optical Shops': 417,
#  'Optical Shops, Eye Doctors': 177,
#  'Other Events': 197,
#  'Other Great Outdoors': 195,
#  'Other Nightlife': 291,
#  'Other Repair Shops': 223,
#  'Paper / Office Supplies Stores': 210,
#  'Parking': 434,
#  'Parks': 26,
#  'Parks, Playgrounds': 134,
#  'Pastry Shops': 87,
#  'Pedestrian Plazas': 307,
#  'Performing Arts Venues': 132,
#  'Pet Services': 465,
#  'Pet Stores': 171,
#  'Pharmacies': 107,
#  'Pharmacies, Drugstores': 420,
#  'Photography Labs': 213,
#  'Photography Studios': 366,
#  'Photography Studios, Photography Labs': 315,
#  'Physical Therapists': 83,
#  'Piers': 89,
#  'Pizza Places': 294,
#  'Pizza Places, Italian Restaurants': 77,
#  'Pizza Places, Restaurants': 267,
#  'Playgrounds': 342,
#  'Plazas': 131,
#  'Police Stations': 399,
#  'Pool Halls': 395,
#  'Pools': 119,
#  'Portuguese Restaurants': 142,
#  'Post Offices': 205,
#  'Print Shops': 84,
#  'Private Schools': 74,
#  'Professional & Other Places': 472,
#  'Public Art': 471,
#  'Pubs': 230,
#  'Racetracks': 292,
#  'Radio Stations': 159,
#  'Ramen Restaurants': 424,
#  'Real Estate Offices': 394,
#  'Record Shops': 32,
#  'Recreation Centers': 460,
#  'Recycling Facilities': 476,
#  'Rental Car Locations': 170,
#  'Residential Buildings (Apartments / Condos)': 94,
#  'Residential Buildings (Apartments / Condos), Buildings': 20,
#  'Residential Buildings (Apartments / Condos), Housing Developments': 265,
#  'Resorts': 124,
#  'Resorts, Hotels': 282,
#  'Rest Areas': 202,
#  'Restaurants': 231,
#  'Restaurants, Cafés': 277,
#  'Rivers': 249,
#  'Rock Clubs': 11,
#  'Sake Bars': 73,
#  'Salad Places': 13,
#  'Salons / Barbershops': 406,
#  'Salons / Barbershops, Health & Beauty Services': 86,
#  'Salons / Barbershops, Nail Salons': 312,
#  'Sandwich Places': 389,
#  'Sandwich Places, Fast Food Restaurants': 252,
#  'Sandwich Places, Fast Food Restaurants, Restaurants': 385,
#  'Scenic Lookouts': 172,
#  'Schools': 281,
#  'Seafood Restaurants': 416,
#  'Shipping Stores': 135,
#  'Shipping Stores, Business Services': 214,
#  'Shoe Stores': 322,
#  'Shopping Malls': 459,
#  'Shopping Malls, Department Stores': 483,
#  'Shopping Plazas': 393,
#  'Shops & Services': 238,
#  'Shrines': 46,
#  'Ski Areas': 35,
#  'Smoke Shops': 156,
#  'Snack Places': 75,
#  'Soba Restaurants': 247,
#  'Soccer Fields': 121,
#  'Soccer Stadiums': 110,
#  'Social Clubs': 470,
#  'Som Tum Restaurants': 4,
#  'Sorority Houses': 254,
#  'Soup Places': 318,
#  'Spanish Restaurants': 404,
#  'Spas': 431,
#  'Speakeasies': 61,
#  'Spiritual Centers': 430,
#  'Sporting Goods Shops': 39,
#  'Sporting Goods Shops, Clothing Stores': 317,
#  'Sports Bars': 398,
#  'Sports Clubs': 354,
#  'Stables': 250,
#  'Stadiums': 28,
#  'Steakhouses': 439,
#  'Storage Facilities': 59,
#  'Street Fairs': 122,
#  'Strip Clubs': 193,
#  'Student Centers': 108,
#  'Supermarkets': 62,
#  'Supermarkets, Discount Stores': 97,
#  'Supermarkets, Grocery Stores': 313,
#  'Surf Spots': 30,
#  'Sushi Restaurants': 443,
#  'Sushi Restaurants, Japanese Restaurants': 138,
#  'TV Stations': 436,
#  'Taco Places': 357,
#  'Tailor Shops': 363,
#  'Tanning Salons': 58,
#  'Tapas Restaurants': 255,
#  'Tattoo Parlors': 173,
#  'Tea Rooms': 485,
#  'Tech Startups': 145,
#  'Temples': 362,
#  'Tennis Courts': 259,
#  'Thai Restaurants': 275,
#  'Theaters': 486,
#  'Theme Park Rides/Attractions': 469,
#  'Theme Parks': 19,
#  'Thrift / Vintage Stores': 346,
#  'Tourist Information Centers': 40,
#  'Toy / Game Stores': 56,
#  'Trade Schools': 225,
#  'Trails': 14,
#  'Train Stations': 8,
#  'Tram Stations': 163,
#  'Transportation Services': 101,
#  'Travel Agencies': 257,
#  'Travel Lounges': 358,
#  'Tunnels': 405,
#  'Turkish Restaurants': 304,
#  'Udon Restaurants': 276,
#  'Universities': 164,
#  'Urgent Care Centers': 445,
#  'Vacation Rentals': 454,
#  'Vegetarian / Vegan Restaurants': 329,
#  'Veterinarians': 92,
#  'Video Game Stores': 327,
#  'Video Stores': 396,
#  'Vietnamese Restaurants': 109,
#  'Warehouse Stores': 488,
#  'Warehouses': 323,
#  'Water Parks': 186,
#  'Waterfalls': 160,
#  'Waterfronts': 411,
#  'Wedding Halls': 149,
#  'Wine Bars': 106,
#  'Wine Shops': 449,
#  'Wineries': 167,
#  'Wings Joints': 54,
#  "Women's Stores": 296,
#  'Yoga Studios': 25,
#  'Zoos': 207}

In [ ]:
sel_cat = {y.lower() : sel_cat[y] for y in sel_cat}

In [ ]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [ ]:
%%cython
def LCS(str S, str T):
    cdef int i, j
    cdef list dp = [[0] * (len(T) + 1) for _ in range(len(S) + 1)]
    for i in range(len(S)):
        for j in range(len(T)):
            dp[i + 1][j + 1] = max(dp[i][j] + (S[i] == T[j]), dp[i + 1][j], dp[i][j + 1], dp[i + 1][j + 1])
    return dp[len(S)][len(T)]

In [ ]:
def post_process(df):
    id2match = dict(zip(df['id'].values, df['matches'].str.split()))

    for base, match in df[['id', 'matches']].values:
        match = match.split()
        if len(match) == 1:        
            continue

        for m in match:
            if base not in id2match[m]:
                id2match[m].append(base)
    df['matches'] = df['id'].map(id2match).map(' '.join)
    return df 

# get manhattan distance
def manhattan(lat1, long1, lat2, long2):
    return np.abs(lat2 - lat1) + np.abs(long2 - long1)

# get haversine distance
def vectorized_haversine(lats1, lats2, longs1, longs2):
    radius = 6371
    dlat=np.radians(lats2 - lats1)
    dlon=np.radians(longs2 - longs1)
    a = np.sin(dlat/2) * np.sin(dlat/2) + np.cos(np.radians(lats1)) \
        * np.cos(np.radians(lats2)) * np.sin(dlon/2) * np.sin(dlon/2)
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    d = radius * c
    return d

In [ ]:
def recall_simple(df):
    threshold = 2
    
    val2id_d = {}
    for col in rec_columns:
        temp_df = df[['id', col]]
        temp_df[col] = temp_df[col].str.lower()
        val2id = temp_df.groupby(col)['id'].apply(set).to_dict()
        val2id_d[col] = val2id
        del val2id
    
    cus_ids = []
    match_ids = []
    for vals in tqdm(df[rec_columns + ['id']].fillna('null').values):
        cus_id = vals[-1]
        match_id = []
        
        rec_match_count = []
        for i in range(len(rec_columns)):
            col = rec_columns[i]
            
            if vals[i] != 'null':
                rec_match_count += list(val2id_d[col][vals[i].lower()])
        rec_match_count = dict(Counter(rec_match_count))
        
        for k, v in rec_match_count.items():
            if v > threshold:
                match_id.append(k)
        
        cus_ids += [cus_id] * len(match_id)
        match_ids += match_id
    
    train_df = pd.DataFrame()
    train_df['id'] = cus_ids
    train_df['match_id'] = match_ids
    train_df = train_df.drop_duplicates()
    del cus_ids, match_ids
    
    num_data = len(train_df)
    num_data_per_id = num_data / train_df['id'].nunique()
    print('Num of data: %s' % num_data)
    print('Num of data per id: %s' % num_data_per_id)
    
    return train_df

In [ ]:
def recall_knn(df, Neighbors = 10, Neighbors2 = 10, Neighbors3 = 10 ):
    print('Start knn grouped by country')
    train_df_country = []
    for country, country_df in tqdm(df.groupby('country')):
        country_df = country_df.reset_index(drop = True)

        neighbors = min(len(country_df), Neighbors)
        knn = KNeighborsRegressor(n_neighbors = neighbors,
                                    metric = 'haversine',
                                    n_jobs = -1)
        knn.fit(country_df[['latitude','longitude']].apply(lambda dr : (dr*math.pi)/180),
                country_df.index)
        dists, nears = knn.kneighbors(
            country_df[['latitude', 'longitude']].apply(lambda dr : (dr*math.pi)/180), 
            return_distance = True)
        # knn.fit(country_df[['latitude','longitude']],
        #         country_df.index)
        # dists, nears = knn.kneighbors(
        #     country_df[['latitude', 'longitude']], 
        #     return_distance = True)

        for k in range(neighbors):            
            cur_df = country_df[['id']]
            cur_df['match_id'] = country_df['id'].values[nears[:, k]]
            cur_df['kdist_country'] = dists[:, k]
            cur_df['kneighbors_country'] = k
            
            train_df_country.append(cur_df)
    train_df_country = pd.concat(train_df_country)
    
    #-------------------------------------------------
    print('Start knn grouped by categories')
    seld = sel_cat

    train_df_categories = []
  
    for categories, categories_df in tqdm(df.groupby('categories')):
        if categories not in seld : continue
        categories_df = categories_df.reset_index(drop = True)

        neighbors = min(len(categories_df), Neighbors2)
        knn = KNeighborsRegressor(n_neighbors = neighbors,
                                    metric = 'haversine',
                                    n_jobs = -1)
        knn.fit(categories_df[['latitude','longitude']].apply(lambda dr : (dr*math.pi)/180),
                categories_df.index)
        dists, nears = knn.kneighbors(
            categories_df[['latitude', 'longitude']].apply(lambda dr : (dr*math.pi)/180), 
            return_distance = True)
        # knn.fit(categories_df[['latitude','longitude']],
        #         categories_df.index)
        # dists, nears = knn.kneighbors(
        #     categories_df[['latitude', 'longitude']], 
        #     return_distance = True)

        for k in range(neighbors):            
            cur_df = categories_df[['id']]
            cur_df['match_id'] = categories_df['id'].values[nears[:, k]]
            cur_df['kdist_categories'] = dists[:, k]
            cur_df['kneighbors_categories'] = k
            cur_df['categories_index'] = seld[categories]
            
            train_df_categories.append(cur_df)
    train_df_categories = pd.concat(train_df_categories)
    #-------------------------------------------------
    print('Start knn')
    train_df = []
    knn = NearestNeighbors(n_neighbors = Neighbors3)
    knn.fit(df[['latitude','longitude']].apply(lambda dr : (dr*math.pi)/180), df.index)
    dists, nears = knn.kneighbors(df[['latitude','longitude']].apply(lambda dr : (dr*math.pi)/180))
    # knn.fit(df[['latitude','longitude']], df.index)
    # dists, nears = knn.kneighbors(df[['latitude','longitude']])
    
    for k in range(Neighbors3):            
        cur_df = df[['id']]
        cur_df['match_id'] = df['id'].values[nears[:, k]]
        cur_df['kdist'] = dists[:, k]
        cur_df['kneighbors'] = k
        train_df.append(cur_df)
    
    train_df = pd.concat(train_df)
    train_df = train_df.merge(train_df_country,
                                 on = ['id', 'match_id'],
                                 how = 'outer')
    #-------------------------------------------------
    train_df = train_df.merge(train_df_categories,
                                on = ['id', 'match_id'],
                                how = 'outer')

    del train_df_country

    pos = train_df['kdist_country'].notna()
    train_df.loc[pos, 'kdist'] = train_df[pos]['kdist_country']
    pos = train_df['kdist_categories'].notna()
    train_df.loc[pos, 'kdist'] = train_df[pos]['kdist_categories']
    sel = ['id', 'match_id', 'kdist', 'kneighbors',
          'kneighbors_country', 'kneighbors_categories','categories_index']
    train_df = train_df[sel]
    # train_df['kneighbors_categories'] = train_df['kneighbors_categories'].astype('category')

    return train_df

In [ ]:
def add_features(df):    
    for col in tqdm(feat_columns):
        if col == 'dist':
            lat = data.loc[df['id']]['latitude'].values
            match_lat = data.loc[df['match_id']]['latitude'].values
            lon = data.loc[df['id']]['longitude'].values
            match_lon = data.loc[df['match_id']]['longitude'].values
            df['latdiff'] = (lat - match_lat)
            df['londiff'] = (lon - match_lon)
            df['manhattan'] = manhattan(lat, lon, match_lat, match_lon)
            df['euclidean'] = (df['latdiff'] ** 2 + df['londiff'] ** 2) ** 0.5
            df['haversine'] = vectorized_haversine(lat, match_lat, lon, match_lon)
            continue
        
        col_values = data.loc[df['id']][col].values.astype(str)
        matcol_values = data.loc[df['match_id']][col].values.astype(str)
        
        if col in vec_columns:
            tv_fit = tfidf_d[col]
            indexs = [id2index_d[i] for i in df['id']]
            match_indexs = [id2index_d[i] for i in df['match_id']]                    
            df[f'{col}_sim'] = tv_fit[indexs].multiply(tv_fit[match_indexs]).\
                                            sum(axis = 1).A.ravel()
        
        geshs = []
        levens = []
        jaros = []
        lcss = []
        fuzwuz = []
        for s, match_s in zip(col_values, matcol_values):
            if s != 'nan' and match_s != 'nan':                    
                geshs.append(difflib.SequenceMatcher(None, s, match_s).ratio())
                levens.append(Levenshtein.distance(s, match_s))
                jaros.append(Levenshtein.jaro_winkler(s, match_s))
                lcss.append(LCS(str(s), str(match_s)))
                fuzwuz.append(fuzz.ratio(s, match_s))
            else:
                geshs.append(np.nan)
                levens.append(np.nan)
                jaros.append(np.nan)
                lcss.append(np.nan)
                fuzwuz.append(np.nan)
        
        df[f'{col}_gesh'] = geshs
        df[f'{col}_leven'] = levens
        df[f'{col}_jaro'] = jaros
        df[f'{col}_lcs'] = lcss
        df[f'{col}_fuz'] = fuzwuz
         
        if col not in ['phone', 'zip']:
            df[f'{col}_len'] = list(map(len, col_values))
            df[f'match_{col}_len'] = list(map(len, matcol_values)) 
            df[f'{col}_len_diff'] = np.abs(df[f'{col}_len'] - df[f'match_{col}_len']) /\
                                        df[f'{col}_len'] 
            df[f'{col}_nleven'] = df[f'{col}_leven'] / \
                                    df[[f'{col}_len', f'match_{col}_len']].max(axis = 1)
            
            df[f'{col}_nlcsk'] = df[f'{col}_lcs'] / df[f'match_{col}_len']
            df[f'{col}_nlcs'] = df[f'{col}_lcs'] / df[f'{col}_len']
            
            df = df.drop(f'{col}_len', axis = 1)
            df = df.drop(f'match_{col}_len', axis = 1)
            gc.collect()
            
    return df

## alphabet convert

In [ ]:
def convert_japanese_alphabet(df: pd.DataFrame):
    kakasi = pykakasi.kakasi()
    kakasi.setMode('H', 'a')  # Convert Hiragana into alphabet
    kakasi.setMode('K', 'a')  # Convert Katakana into alphabet
    kakasi.setMode('J', 'a')  # Convert Kanji into alphabet
    conversion = kakasi.getConverter()

    def convert(row):
        for column in ["name", "address", "city", "state"]:
            try:
                row[column] = conversion.do(row[column])
            except:
                pass
        return row

    df[df["country"] == "JP"] = df[df["country"] == "JP"].progress_apply(convert, axis=1)
    return df

## for CV

In [ ]:
# https://www.kaggle.com/code/columbia2131/foursquare-iou-metrics
def get_id2poi(input_df: pd.DataFrame) -> dict:
  return dict(zip(input_df['id'], input_df['point_of_interest']))

def get_poi2ids(input_df: pd.DataFrame) -> dict:
  return input_df.groupby('point_of_interest')['id'].apply(set).to_dict()

# KNNで選ばれたデータの正解のカバー率を計測する
def get_score(input_df: pd.DataFrame):
  scores = []
  for id_str, matches in zip(input_df['id'].to_numpy(), input_df['matches'].to_numpy()):
    targets = poi2ids[id2poi[id_str]]
    preds = set(matches.split())
    score = len((targets & preds)) / len((targets | preds))
    scores.append(score)
  scores = np.array(scores)
  return scores.mean()

In [ ]:
cdict =  {'ATMs': 'Banks',
      'Airport Services': 'Airports',
      'Airport Terminals': 'Airports',
      'American Restaurants': 'Fast Food Restaurants',
      'Asian Restaurants': 'Chinese Restaurants',
      'Auto Dealerships': 'Automotive Shops',
      'Auto Dealerships, Automotive Shops': 'Auto Dealerships',
      'Auto Workshops, Automotive Shops': 'Automotive Shops',
      'Auto Workshops, Automotive Shops, Auto Garages': 'Automotive Shops',
      'Automotive Shops, Auto Dealerships': 'Automotive Shops',
      'Automotive Shops, Auto Workshops, Auto Garages': 'Automotive Shops',
      'Banks, ATMs': 'Banks',
      'Banks, Buildings': 'Banks',
      'Banks, Offices': 'Banks',
      'Beaches, Surf Spots': 'Beaches',
      'Bed & Breakfasts': 'Hotels',
      'Boutiques': 'Clothing Stores',
      'Buddhist Temples': 'Temples',
      'Buildings': 'Offices',
      'Buildings, Offices': 'Offices',
      'Buildings, Residential Buildings (Apartments / Condos)': 'Residential Buildings (Apartments / Condos)',
      'Burger Joints': 'Fast Food Restaurants',
      'Bus Stations': 'Bus Stops',
      'Cafés, Coffee Shops': 'Cafés',
      'Coffee Shops': 'Cafés',
      'Coffee Shops, Cafés': 'Cafés',
      'College Academic Buildings': 'Universities',
      'College Auditoriums': 'College Classrooms',
      'Coworking Spaces': 'Offices',
      'Department Stores': 'Shopping Malls',
      "Doctor's Offices": 'Hospitals',
      'Drugstores': 'Pharmacies',
      'Fast Food Restaurants, Burger Joints': 'Fast Food Restaurants',
      'Fast Food Restaurants, Fried Chicken Joints': 'Fast Food Restaurants',
      'Fried Chicken Joints': 'Fast Food Restaurants',
      'Fried Chicken Joints, Fast Food Restaurants': 'Fried Chicken Joints',
      'Gas Stations, Convenience Stores': 'Gas Stations',
      'Government Buildings': 'Offices',
      'Grocery Stores': 'Convenience Stores',
      'Grocery Stores, Supermarkets': 'Grocery Stores',
      'Gyms or Fitness Centers': 'Gyms',
      'Gyms or Fitness Centers, Gyms': 'Gyms',
      'High Schools': 'College Classrooms',
      'Hospital Wards': 'Hospitals',
      'Hospitals, Medical Centers': 'Hospitals',
      'Hotels': 'Residential Buildings (Apartments / Condos)',
      'Hotels, Resorts': 'Hotels',
      'Housing Developments': 'Residential Buildings (Apartments / Condos)',
      'Housing Developments, Residential Buildings (Apartments / Condos)': 'Residential Buildings (Apartments / Condos)',
      'Ice Cream Shops, Dessert Shops': 'Ice Cream Shops',
      'Indonesian Restaurants': 'Chinese Restaurants',
      'Italian Restaurants': 'Restaurants',
      'Italian Restaurants, Pizza Places': 'Italian Restaurants',
      'Light Rail Stations': 'Metro Stations',
      'Medical Centers': "Doctor's Offices",
      "Medical Centers, Doctor's Offices": "Doctor's Offices",
      'Medical Centers, Hospitals': 'Hospitals',
      'Middle Schools': 'College Classrooms',
      'Mobile Phone Shops, Business Services': 'Mobile Phone Shops',
      'Mobile Phone Shops, Electronics Stores': 'Mobile Phone Shops',
      'Motels': 'Residential Buildings (Apartments / Condos)',
      'Movie Theaters, Multiplexes': 'Movie Theaters',
      'Multiplexes': 'Movie Theaters',
      'Multiplexes, Movie Theaters': 'Movie Theaters',
      'Noodle Houses': 'Chinese Restaurants',
      'Offices': 'Banks',
      'Offices, Buildings': 'Offices',
      'Other Great Outdoors': 'Parks',
      'Pizza Places, Italian Restaurants': 'Pizza Places',
      'Pizza Places, Restaurants': 'Pizza Places',
      'Pubs': 'Bars',
      'Ramen Restaurants': 'Noodle Houses',
      'Residential Buildings (Apartments / Condos), Buildings': 'Offices',
      'Residential Buildings (Apartments / Condos), Housing Developments': 'Residential Buildings (Apartments / Condos)',
      'Resorts': 'Hotels',
      'Salons / Barbershops, Health & Beauty Services': 'Salons / Barbershops',
      'Salons / Barbershops, Nail Salons': 'Salons / Barbershops',
      'Sandwich Places, Fast Food Restaurants': 'Sandwich Places',
      'Schools': 'College Classrooms',
      'Shopping Malls': 'Offices',
      'Soccer Stadiums': 'Soccer Fields',
      'Supermarkets': 'Grocery Stores',
      'Supermarkets, Discount Stores': 'Grocery Stores',
      'Supermarkets, Grocery Stores': 'Grocery Stores',
      'Sushi Restaurants': 'Japanese Restaurants',
      'Tech Startups': 'Offices',
      'Thai Restaurants': 'Restaurants',
      'Train Stations': 'Metro Stations',
      'Universities': 'College Classrooms',
      "Women's Stores": 'Clothing Stores'}

In [ ]:
# cdict = {y.lower() : cdict[y].lower() for y in cdict}

In [ ]:
def f(x) :
  if type(x)==float and math.isnan(x) : return x
  if x in cdict : return cdict[x]
  return x

In [ ]:
from sklearn.model_selection import GroupKFold

## Dada process
if is_colab:
    data = pd.read_csv('../input/foursquare-location-matching/train.csv')
    tv_ids_d = np.load('/content/drive/MyDrive/datas/foursquare/traindata_cat2/tv_ids_d.npy', allow_pickle=True).item()
    data = data.set_index('id')
    data = data.loc[tv_ids_d['valid_ids']]
    kf = GroupKFold(n_splits=25)
    for trn_idx, val_idx in kf.split(data, 
                                      data['point_of_interest'], 
                                      data['point_of_interest']):
      idx = val_idx
      break
    data = data.iloc[idx] 
    data = data.reset_index()
    # id --> poi変換用の辞書. １対１の関係
    id2poi = get_id2poi(data)
    # poi --> id変換用の辞書辞書. １対多の関係
    poi2ids = get_poi2ids(data)   
    data = data.drop('point_of_interest', axis = 1)

else :
    data = pd.read_csv('../input/foursquare-location-matching/test.csv')

    if len(data) < 20:
        data = pd.read_csv('../input/foursquare-location-matching/train.csv',
                          nrows = 100)
        data = data.drop('point_of_interest', axis = 1)
   
#need to new dataset
# data = convert_japanese_alphabet(data)
data['categories2'] = data['categories'].apply(f)

# special process
def get_lower(x):
    try:
        return x.lower()
    except:
        return x

for col in data.columns:
    if data[col].dtype == object and col != 'id':
        data[col] = data[col].apply(get_lower)
        
id2index_d = dict(zip(data['id'].values, data.index))

tfidf_d = {}
for col in vec_columns:
    tfidf = TfidfVectorizer()
    tv_fit = tfidf.fit_transform(data[col].fillna('nan'))
    tfidf_d[col] = tv_fit

out_df = pd.DataFrame()
out_df['id'] = data['id'].unique().tolist()
out_df['match_id'] = out_df['id']

test_data_simple = recall_simple(data)
test_data = recall_knn(data, num_neighbors, num_neighbors2, num_neighbors3)

print('train data by knn: %s' % len(test_data))
test_data = test_data.merge(test_data_simple,
                             on = ['id', 'match_id'],
                             how = 'outer')
del test_data_simple
gc.collect()

data = data.set_index('id')
print('Num of unique id: %s' % test_data['id'].nunique())
print('Num of test data: %s' % len(test_data))
print(test_data.sample(5))

  0%|          | 0/22777 [00:00<?, ?it/s]

Num of data: 19918
Num of data per id: 1.3478143185816756
Start knn grouped by country


  0%|          | 0/127 [00:00<?, ?it/s]

Start knn grouped by categories


  0%|          | 0/3329 [00:00<?, ?it/s]

Start knn
train data by knn: 575123
Num of unique id: 22777
Num of test data: 577782
                      id          match_id     kdist  kneighbors  \
246536  E_d191effdac4d76  E_70dde168ab3460  0.008469        10.0   
122197  E_5c113bcdabb5e1  E_2272c9238422e8  0.000184         5.0   
215670  E_76b7cc14e945c2  E_88a1a6a4bf4cf7  0.001019         9.0   
404325  E_bf07b4dc67cb41  E_1cef732913cde8  0.022621        17.0   
79002   E_76a5eec29e04c8  E_cb6629e9f1eed9  0.000640         3.0   

        kneighbors_country  kneighbors_categories  categories_index  
246536                 NaN                    NaN               NaN  
122197                 5.0                    NaN               NaN  
215670                 NaN                    NaN               NaN  
404325                 NaN                    NaN               NaN  
79002                  3.0                    NaN               NaN  


In [ ]:
data.sample(20)

,name,latitude,longitude,address,city,state,zip,country,url,phone,categories,categories2
id,,,,,,,,,,,,
E_d84eb03f2b3435,studio 3 lapiaza 21cinpelex,-6.160616,106.906242,NaN,jakarta capital region,jakarta capital region,NaN,id,NaN,NaN,NaN,NaN
E_768b1b7e0d43e3,millennium hotel paris charles de gaulle,49.005671,2.520554,2 allée du verger,paris,ãle-de-france,95700,fr,https://www.millenniumhotels.com/en/paris/mill...,134293333,hotels,residential buildings (apartments / condos)
E_fe6c99779da35e,hotel viewport,-34.903530,-56.198998,uruguay 825,montevideo,NaN,NaN,uy,NaN,NaN,hotels,residential buildings (apartments / condos)
E_9a6a7ec0ad6b71,canary wireless,33.945389,-117.454190,6300 van buren boulevard,riverside,ca,92503,us,NaN,9513433615,mobile phone shops,mobile phone shops
E_59706f4ed2b824,автозапчасти,55.792235,37.603946,новосущевская ул.,москва,NaN,NaN,ru,NaN,NaN,automotive shops,automotive shops
E_ec32c092895c62,172a edgedale plains,1.395157,103.911632,172a edgedale plains,NaN,NaN,NaN,sg,NaN,NaN,residential buildings (apartments / condos),residential buildings (apartments / condos)
E_dce84df1c9674f,kafe sakae.西餐廳,3.098252,101.737321,jalan siput akek,kuala lumpur,wp kuala lumpur,NaN,my,NaN,NaN,steakhouses,steakhouses
E_43c46b6fab9d6f,time warner cable,41.367550,-82.108147,525 broad st,elyria,oh,44035,us,NaN,(440) 681-8652,business services,business services
E_72b18dd4413490,pokinometry,34.101669,-118.340410,6801 hollywood blvd #316,los angeles,calif,90028,us,http://pokinometry.com,NaN,poke places,poke places


In [ ]:
# add_features(test_data[:1000])[['categories_lcs','categories2_lcs']].sample(50)

In [ ]:
## Model load
mtype = [
    'lightgbm',
    'lightgbm',
    'lightgbm',
    'lightgbm',
    'lightgbm',
    # 'catboost',
]
models = [
    '/content/drive/MyDrive/datas/foursquare/lightgbm/lgbm_cat2_fold0.txt',
    '/content/drive/MyDrive/datas/foursquare/lightgbm/lgbm_cat2_fold1.txt',
    '/content/drive/MyDrive/datas/foursquare/lightgbm/lgbm_cat2_fold2.txt',
    '/content/drive/MyDrive/datas/foursquare/lightgbm/lgbm_cat2_fold3.txt',
    '/content/drive/MyDrive/datas/foursquare/lightgbm/lgbm_cat2_fold4.txt',
    # '/content/drive/MyDrive/datas/foursquare/lightgbm/lgbm_fold0.txt',
    # '/content/drive/MyDrive/datas/foursquare/lightgbm/lgbm_w352_fold0.txt',
    # '/content/drive/MyDrive/datas/foursquare/lightgbm/cat4_fold0.cbm',
    # '/content/drive/MyDrive/datas/foursquare/lightgbm/lgbm_w200_fold0.txt',
]
features = [
           TRAIN_FEATURES,
           TRAIN_FEATURES,
           TRAIN_FEATURES,
           TRAIN_FEATURES,
           TRAIN_FEATURES,
           ]

mix_model = []
for i, e in enumerate(mtype) :
    if e == 'catboost' :
      m = CatBoostClassifier()
      m.load_model(models[i])
      mix_model.append(m)
    else :
      mix_model.append(lgb.Booster(model_file = models[i]))

mix_model

In [ ]:
%%time
## Prediction
count = 0
start_row = 0
pred_df = pd.DataFrame()
unique_id = test_data['id'].unique().tolist()
num_split_id = len(unique_id) // NUM_SPLIT
for k in range(1, NUM_SPLIT + 1):
    print('Current split: %s' % k)
    end_row = start_row + num_split_id
    if k < NUM_SPLIT:
        cur_id = unique_id[start_row : end_row]
        cur_data = test_data[test_data['id'].isin(cur_id)]
    else:
        cur_id = unique_id[start_row: ]
        cur_data = test_data[test_data['id'].isin(cur_id)]
    
    # add features & model prediction
    cur_data = add_features(cur_data)

    # print(cur_data.head())
    # cur_data['kdist_diff'] = (cur_data['kdist'] - cur_data['kdist_country']) /\
    #                             cur_data['kdist_country']
    # cur_data['kneighbors_mean'] = cur_data[['kneighbors', 'kneighbors_country']].mean(axis = 1)

    preds = []
    for num, model in enumerate(mix_model) :
      if mtype[num] == 'catboost' :
          res = model.predict_proba(cur_data[features[num]])[:,1]
      else :
          res = model.predict(cur_data[features[num]])#,predict_disable_shape_check=True)
      cur_data['pred{}'.format(num)] = res
      preds.append('pred{}'.format(num))

#    cur_pred_df = cur_data[cur_data['pred'] > THRESHOLD][['id', 'match_id']]
    cur_pred_df = cur_data[['id','match_id']+preds]
    pred_df = pd.concat([pred_df, cur_pred_df])
    
    start_row = end_row
    count += len(cur_data)

    del cur_data, cur_pred_df
    gc.collect()
print(count)

Current split: 1


  0%|          | 0/11 [00:00<?, ?it/s]

Current split: 2


  0%|          | 0/11 [00:00<?, ?it/s]

Current split: 3


...

Current split: 9


  0%|          | 0/11 [00:00<?, ?it/s]

Current split: 10


  0%|          | 0/11 [00:00<?, ?it/s]

577782
CPU times: user 29min 14s, sys: 4.15 s, total: 29min 18s
Wall time: 17min 43s


In [ ]:
pred_df

,id,match_id,pred0,pred1,pred2,pred3,pred4
0,E_0001e184ce8606,E_0001e184ce8606,1.000000e+00,1.000000e+00,1.000000,1.000000,1.000000e+00
1,E_000292dce833cc,E_000292dce833cc,9.999988e-01,9.999991e-01,0.999998,0.999997,9.999968e-01
2,E_000cad8ab4d1df,E_000cad8ab4d1df,9.999999e-01,9.999999e-01,1.000000,1.000000,9.999999e-01
3,E_000d9110deea1d,E_000d9110deea1d,9.999999e-01,9.999990e-01,0.999999,0.999999,9.999994e-01
4,E_001b8579ed742e,E_001b8579ed742e,9.999993e-01,9.999976e-01,0.999997,0.999997,9.999966e-01
...,...,...,...,...,...,...,...
577777,E_faaac6bd1a8371,E_dac225bd95c74a,5.775012e-06,5.149520e-07,0.000002,0.000001,2.591655e-07
577778,E_faaac6bd1a8371,E_df455509331d85,1.708026e-04,1.735667e-05,0.000028,0.000006,7.844042e-05
577779,E_fc657cdc4ea491,E_7b79e99e27c704,4.294385e-06,1.141693e-04,0.000010,0.000010,1.296298e-05
577780,E_fc657cdc4ea491,E_f05566fad78c02,4.294385e-06,1.141693e-04,0.000010,0.000010,1.296298e-05


In [ ]:
pred_df['pred']=(pred_df['pred0'] + pred_df['pred1'] + pred_df['pred2'] + pred_df['pred3'] + pred_df['pred4'])/5
# pred_df['pred']=(pred_df['pre d0']+pred_df['pred1']+pred_df['pred2'])/3
# pred_df['pred']=(pred_df['pred0']+pred_df['pred1'])/2
# pred_df['pred']=pred_df['pred0']

In [ ]:
pred_df

,id,match_id,pred0,pred1,pred2,pred3,pred4,pred
0,E_0001e184ce8606,E_0001e184ce8606,1.000000e+00,1.000000e+00,1.000000,1.000000,1.000000e+00,1.000000
1,E_000292dce833cc,E_000292dce833cc,9.999988e-01,9.999991e-01,0.999998,0.999997,9.999968e-01,0.999998
2,E_000cad8ab4d1df,E_000cad8ab4d1df,9.999999e-01,9.999999e-01,1.000000,1.000000,9.999999e-01,1.000000
3,E_000d9110deea1d,E_000d9110deea1d,9.999999e-01,9.999990e-01,0.999999,0.999999,9.999994e-01,0.999999
4,E_001b8579ed742e,E_001b8579ed742e,9.999993e-01,9.999976e-01,0.999997,0.999997,9.999966e-01,0.999997
...,...,...,...,...,...,...,...,...
577777,E_faaac6bd1a8371,E_dac225bd95c74a,5.775012e-06,5.149520e-07,0.000002,0.000001,2.591655e-07,0.000002
577778,E_faaac6bd1a8371,E_df455509331d85,1.708026e-04,1.735667e-05,0.000028,0.000006,7.844042e-05,0.000060
577779,E_fc657cdc4ea491,E_7b79e99e27c704,4.294385e-06,1.141693e-04,0.000010,0.000010,1.296298e-05,0.000030
577780,E_fc657cdc4ea491,E_f05566fad78c02,4.294385e-06,1.141693e-04,0.000010,0.000010,1.296298e-05,0.000030


In [ ]:
tmp = pred_df

In [ ]:
pred_df = pred_df[pred_df['pred'] > THRESHOLD][['id', 'match_id']]
pred_df

,id,match_id
0,E_0001e184ce8606,E_0001e184ce8606
1,E_000292dce833cc,E_000292dce833cc
2,E_000cad8ab4d1df,E_000cad8ab4d1df
3,E_000d9110deea1d,E_000d9110deea1d
4,E_001b8579ed742e,E_001b8579ed742e
...,...,...
568254,E_f6eaf90b336cdd,E_0851915e48629e
577704,E_ec968c3cab8a5a,E_2723056dc01109
577705,E_ece61d65c64ce9,E_d2909a10c48fdd
577706,E_eda0c9994a0faf,E_eda0c9994a0faf


In [ ]:
## Submission    
out_df = pd.concat([out_df, pred_df])
out_df = out_df.groupby('id')['match_id'].\
                        apply(list).reset_index()
out_df['matches'] = out_df['match_id'].apply(lambda x: ' '.join(set(x)))
out_df = post_process(out_df)
print('Unique id: %s' % len(out_df))
print(out_df.head())

out_df[['id', 'matches']].to_csv('submission.csv', index = False)

Unique id: 22777
                 id                                           match_id  \
0  E_0001e184ce8606  [E_0001e184ce8606, E_0001e184ce8606, E_e9df025...   
1  E_000292dce833cc  [E_000292dce833cc, E_000292dce833cc, E_3f005f0...   
2  E_000cad8ab4d1df  [E_000cad8ab4d1df, E_000cad8ab4d1df, E_fbea1a8...   
3  E_000d9110deea1d               [E_000d9110deea1d, E_000d9110deea1d]   
4  E_001b8579ed742e               [E_001b8579ed742e, E_001b8579ed742e]   

                             matches  
0  E_e9df025ab2137d E_0001e184ce8606  
1  E_000292dce833cc E_3f005f05c5c146  
2  E_fbea1a8a91a4bb E_000cad8ab4d1df  
3                   E_000d9110deea1d  
4                   E_001b8579ed742e  


## show CV

In [ ]:
if is_colab:
  print(f"CV: {get_score(out_df):.6f}")

CV: 0.913176


In [ ]:
# CV: 0.914089
# CV: 0.913176 5fold

In [ ]:
# CV: 0.914605 no weight no kanji(miss)
#------------------------------------------------------------------
# CV: 0.934487 weighted sqrt(neg/pos) no kanji(miss)
# CV: 0.934492 weighted srqt(neg/pos) 2fold no kanji(miss)
# CV: 0.931351 wdithged sqrt(neg/pos) light version no kanji(miss)
# CV: 0.934244 weighted sqrt(neg/pos) 
# CV: 0.934954 weighted sqrt(neg/pos) middle version kanji 

In [ ]:
# CV: 0.907733　LB:0.856

In [ ]:
# CV: 0.907162 1fold
# CV: 0.911148 1fold weighted
# CV : 0.909899 2fold (non weight + weighted)
# CV: 0.909890 1fold weight = 2
